---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession_start/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states_name = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
state_set = set(states_name.values())

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    data = open('university_towns.txt', 'r').read().split('\n')
    data = list(filter(None, data))
    slist = []
    states = []
    for index, value in enumerate(data):
        v = value.split('[')[0]
        if v in state_set:
            slist.append(index)
            states.append(v)
        else:
            states.append(states[-1])
            
    # remove rows of state name
    data = [x.split(' (')[0] for x in data]
    data = np.delete(data,(np.r_[slist]))
    states = np.delete(states,(np.r_[slist]))
    
    uni_df = pd.DataFrame(np.array([states, data]).T, columns=['State', 'RegionName'])
    return uni_df
get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [6]:
GDP = pd.read_excel('gdplev.xls').iloc[219:,[4,6]]
GDP = GDP.rename(columns={'Unnamed: 4':'quarter', 'Unnamed: 6':'value'})
GDP = GDP.reset_index().loc[:,['quarter','value']]

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    recession = []
    for i in range (2,len(GDP)-1):
        if (GDP.iloc[i,1] < GDP.iloc[i-1,1]) & (GDP.iloc[i-1,1] < GDP.iloc[i-2,1]):
            recession.append(GDP.iloc[i-2,0])
    return recession[1]
get_recession_start()

'2008q3'

In [7]:
start = GDP[GDP['quarter'] == get_recession_start()].index[0]

def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    recession = []
    for i in range (start,len(GDP)-1):
        if (GDP.iloc[i,1] > GDP.iloc[i-1,1]) & (GDP.iloc[i-1,1] > GDP.iloc[i-2,1]):
            recession.append(GDP.iloc[i,0])
    return recession[0]       
get_recession_end()

'2009q4'

In [8]:
end = GDP[GDP['quarter'] == get_recession_end()].index[0]

def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    bottom = GDP.iloc[end-2,0]
    return bottom
get_recession_bottom()

'2009q2'

In [9]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    home = pd.read_csv('City_Zhvi_AllHomes.csv').iloc[:,np.r_[2,1,51:251]]
    home = home.set_index('State').rename(index = states_name)
    home = home.reset_index().sort_values(by=['State']).set_index(['State', 'RegionName'])
    home_g = home.groupby(pd.PeriodIndex(home.columns, freq='Q'), axis=1).mean()
    col = [str(home_g.columns[i]).lower() for i in range(len(home_g.columns))]
    home_g.columns = col
    return home_g
convert_housing_data_to_quarters().head()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Mulga                  NaN            NaN            NaN   
        Trussville   164400.000000  165900.000000  167600.000000   
        Satsuma       88600.000000   90966.666667   90333.333333   
        Irvington     72466.666667   74100.000000   75700.000000   
        Mount Olive  106500.000000  107333.333333  106966.666667   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Mulga                  NaN            NaN            NaN   
        Trussville   169766.666667  172533.333333  173433.333333   
        Satsuma       89600.000000   90233.333333   90766.666667   
        Irvington     76266.666667   76300.000000   74300.000000   
        Mount Olive  107200.000000  108266.666667  108766.666667   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Mulga                  NaN            NaN            NaN   
        Trussville   174166.666667  176700.000000  178000.000000   
        Satsuma       90633.333333   92200.000000   93000.000000   
        Irvington     73500.000000   73833.333333   75966.666667   
        Mount Olive  110166.666667  112400.000000  113966.666667   

                            2002q2    ...            2014q2         2014q3  \
State   RegionName                    ...                                    
Alabama Mulga                  NaN    ...      45466.666667   43600.000000   
        Trussville   180800.000000    ...     212033.333333  210966.666667   
        Satsuma       93200.000000    ...     125266.666667  127500.000000   
        Irvington     79266.666667    ...      90533.333333   95666.666667   
        Mount Olive  115000.000000    ...     132066.666667  132366.666667   

                            2014q4         2015q1         2015q2  \
State   RegionName                                                 
Alabama Mulga         42166.666667   41933.333333   42733.333333   
        Trussville   214566.666667  216600.000000  220066.666667   
        Satsuma      128266.666667  129266.666667  131300.000000   
        Irvington     98466.666667   99500.000000   98800.000000   
        Mount Olive  132066.666667  135433.333333  138100.000000   

                            2015q3         2015q4         2016q1  \
State   RegionName                                                 
Alabama Mulga         42033.333333   43033.333333   40666.666667   
        Trussville   219833.333333  219433.333333  220266.666667   
        Satsuma      133333.333333  134800.000000  134666.666667   
        Irvington     97600.000000   98000.000000  101500.000000   
        Mount Olive  138433.333333  140633.333333  142866.666667   

                            2016q2    2016q3  
State   RegionName                            
Alabama Mulga         38366.666667   38300.0  
        Trussville   223566.666667  225600.0  
        Satsuma      133533.333333  133100.0  
        Irvington    103966.666667  103950.0  
        Mount Olive  144033.333333  143850.0  

[5 rows x 67 columns]

In [43]:
home_r = convert_housing_data_to_quarters()

startq = get_recession_start()
startq = home_r.columns[home_r.columns.get_loc(startq)-1]
bottomq = get_recession_bottom()

def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    home_total = pd.DataFrame(home_r[startq]/home_r[bottomq],columns = ['ratio'])
    #home_r_no_dup = home_r.copy().reset_index().drop(6537).set_index(['State', 'RegionName']) # remove another CLinton, NY
    #home_total_no_dup = pd.DataFrame(home_r_no_dup[startq]/home_r_no_dup[bottomq],columns = ['ratio'])
    
    uni = get_list_of_university_towns()
    uni_lst = list(uni.itertuples(index=False, name=None))
    
    home_uni = home_total.loc[home_total.index.isin(uni_lst)]
    #home_uni = home_total_no_dup.loc[home_total_no_dup.index.isin(uni_lst)].reset_index()
    home_non_uni = home_total.loc[~(home_total.index.isin(uni_lst))]
    # or df = pd.merge(home_total.reset_index(), uni, on = uni.columns.tolist(), indicator = flag, how =  outer)
    # home_uni = df[df['_flag'] == 'both']
    # home_non_uni = df[df['_flag'] !== 'both']
    
    p = ttest_ind(home_uni['ratio'], home_non_uni['ratio'], nan_policy='omit')[1]
    different = (p < 0.01)
    if home_non_uni['ratio'].mean() <  home_non_uni['ratio'].mean():
        better = 'non-university town'
    else: better = 'university town'
    return (different, p, better)
run_ttest()

(True, 0.0027240637047531249, 'university town')